# **This is the assignment for Segmenting and Clustering Neighborhoods in Toronto**

#### Following Cell contains the code to import and install all the required packages in this assignment

In [1]:
#!conda install --channel conda-forge geopandas geoplot geopy --yes
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install --channel conda-forge geopandas geoplot geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


In [2]:
%pip install beautifulsoup4
%pip install lxml
%pip install html5lib

# **This is start of First part of the assignment.**

#### Following cell contains the code to create the pandas dataframe by scraping the webpage as per the instructions given in the assignment.

In [3]:
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
raw_df=df[0]
raw_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


####In the following cell raw dataframe has been filtered to remove the rows having 'Borough' = 'Not assigned'

In [4]:
filter_df=raw_df[raw_df['Borough'] != 'Not assigned']
filter_df=filter_df.reset_index(drop=True)
filter_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### In the following cell we have checked if there exists multiple entries for Postal Code

In [5]:
filter_df.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M8Y,North York,Downsview
freq,1,24,4


### From the describe result it is concluded that there are no duplicate entries for Postal Code. Reason - Total Postal Codes count is same as unique Postal Codes. Hence there is no need of converting multiple Postal Codes records into single rows

#### In the Following cell we are checking if there exists records with proper Borough value and Neighborhood value set to 'Not assigned'

In [6]:
filter_df[filter_df['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


#### From the test above it is concluded that there are no records in the dataframe having 'Neighborhood' column with value 'Not assigned'. Hence there is no requirement of replacing 'Not assigned' value in ''Neighborhood' column with the value in 'Borough' column

#### Following Cell shows the shape of the final dataframe as per the instructions given

<a id='item1'></a>

In [7]:
filter_df.shape

(103, 3)

# **First part of the assignment Ends here**

# **Second part of the assignment Starts here**

### IN the following cell we are creating the dataframe for various Postal Codes from the csv file link provided in the instructions of the assignment.

In [8]:
import pandas as pd
df_latlong_data = pd.read_csv('https://cocl.us/Geospatial_data')
df_latlong_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### In the following cell we are merging the two dataframes : one with postal codes and neighborhood details and the other with postal codes and lat-long details. The merging is done on 'Postal code' column

In [9]:
final_df = pd.merge(filter_df, df_latlong_data, how='inner', on=['Postal Code'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

final_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##### In the following cell we have renamed the column 'Postal Code' to 'PostalCode' as per the requirement given in instructions

In [10]:
final_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
final_df.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# **Second part of the assignment Ends here**

# **Third part of the assignment Starts here**

In [11]:
final_df['cont_toronto']=final_df['Borough'].str.contains('Toronto')
final_df.head()
final_df=final_df[final_df['cont_toronto'] == True ]
final_df=final_df.filter(['PostalCode','Borough','Neighborhood','Latitude','Longitude'])
final_df=final_df.reset_index(drop=True)
final_df.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
print(final_df['Borough'].unique())

['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


## Explore and cluster the neighborhoods in Toronto (Condition - Only neighborhoods with Borough having word 'Toronto' considered)

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(final_df['Borough'].unique()),
        final_df.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


##Generate map to display neighborhoods in Toronto and how they are clustered together

In [14]:
#naming agent Toronto_explorer and finding the lat/long for Toronto
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="explore_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


The geographical coordinates of Toronto are 43.6534817, -79.3839347.


### In the following cell we hve sliced the final_df dataframe and create a new dataframe for 'Downtown Toronto' data

In [15]:
downtown_data = final_df[final_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [16]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="explore_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


##Generate map to display neighborhoods in Downtown Toronto and how they are clustered together

In [17]:
# create map of downtown using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = '1BMK0S0BM3WY0AYZDHI3LGVTPHBALBFWQG0U14W20T1TQNIP' 
CLIENT_SECRET = 'UZDRGEAKSNAXCYN54101V3NQ3U0KYPZ34QDUN2XTRAV3GQFB' 
VERSION = '20200606' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1BMK0S0BM3WY0AYZDHI3LGVTPHBALBFWQG0U14W20T1TQNIP
CLIENT_SECRET:UZDRGEAKSNAXCYN54101V3NQ3U0KYPZ34QDUN2XTRAV3GQFB


# Explore neighborhood in Downtown Toronto

In [19]:
downtown_data.loc[5, 'Neighborhood']

'St. James Town'

### Get the neighborhood's latitude and longitude values

In [20]:
neighborhood_no=5
neighborhood_latitude = downtown_data.loc[neighborhood_no, 'Latitude'] 
neighborhood_longitude = downtown_data.loc[neighborhood_no, 'Longitude'] 
neighborhood_name = downtown_data.loc[neighborhood_no, 'Neighborhood'] 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


## Get the top 100 venues that are in St. James Town within a radius of 600 meters.

#### We are intrested in Mexican Food and want to search the restaurants within 600 meters of St. James Town 

In [21]:
latitude=neighborhood_latitude
longitude=neighborhood_longitude
search_query = 'Mexican'
radius = 600
LIMIT=30
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1BMK0S0BM3WY0AYZDHI3LGVTPHBALBFWQG0U14W20T1TQNIP&client_secret=UZDRGEAKSNAXCYN54101V3NQ3U0KYPZ34QDUN2XTRAV3GQFB&ll=43.6514939,-79.3754179&v=20200606&query=Mexican&radius=600&limit=30'

####Send the GET Request and examine the results

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ede52085fb726001bb0ed1b'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/embassy_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d12c951735',
      'name': 'Embassy / Consulate',
      'pluralName': 'Embassies / Consulates',
      'primary': True,
      'shortName': 'Embassy'}],
    'hasPerk': False,
    'id': '4f7d9a0ee4b01f9f43bd0603',
    'location': {'address': '11 King St',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 386,
     'formattedAddress': ['11 King St', 'Toronto ON', 'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64894155562399,
       'lng': -79.37867590887817}],
     'lat': 43.64894155562399,
     'lng': -79.37867590887817,
     'state': 'ON'},
    'name': 'Mexican Consulate',
    'referralId': 'v-1591628090'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexica

In [23]:
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f7d9a0ee4b01f9f43bd0603,Mexican Consulate,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1591628090,False,11 King St,43.648942,-79.378676,"[{'label': 'display', 'lat': 43.64894155562399...",386,CA,Toronto,ON,Canada,"[11 King St, Toronto ON, Canada]",NaN,NaN
1,4ae623e3f964a52002a521e3,Quesada Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1591628090,False,92 King Street E,43.649939,-79.374496,"[{'label': 'display', 'lat': 43.64993863703611...",188,CA,Toronto,ON,Canada,"[92 King Street E (at Church St), Toronto ON M...",at Church St,M5C 2V8
2,4d76c9eec95f5941ca6e2321,Chimichanga Mexican Grill,[],v-1591628090,False,70 Yorkville Ave,43.651772,-79.379482,"[{'label': 'display', 'lat': 43.65177154540889...",328,CA,Toronto,ON,Canada,"[70 Yorkville Ave (Yorkville/Bel Air), Toronto...",Yorkville/Bel Air,M5R 1B9
3,4bdef9836198c9b6de8115ff,Mucho Burrito Fresh Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1591628090,False,333 Bay St.,43.650661,-79.379401,"[{'label': 'display', 'lat': 43.65066054666582...",333,CA,Toronto,ON,Canada,"[333 Bay St. (Bay Adelaide Centre), Toronto ON...",Bay Adelaide Centre,M5H 2R2
4,4be80776c5220f479432a9ca,Mucho Burrito Fresh Mexican Grill,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1591628090,False,"181 Bay St,",43.646890,-79.378681,"[{'label': 'display', 'lat': 43.64689, 'lng': ...",575,CA,Toronto,ON,Canada,"[181 Bay St, (Brookfield Place), Toronto ON M5...",Brookfield Place,M5J 2T3


In [24]:
print('There are {} venues near St. James Town where we can get Mexican Food.'.format(dataframe.shape[0]))

There are 5 venues near St. James Town where we can get Mexican Food.


## Search results: From the search Foursquare API has returned details of 5 Venues within 600 meters of St. James Town where we can get Mexican Food

## Now we want to explore the venues near restaurant 'Quesada Mexican Grill' within 700 meters

In [25]:
venue_no=0
venue_latitude = dataframe.loc[venue_no, 'location.lat'] 
venue_longitude = dataframe.loc[venue_no, 'location.lng'] 
venue_name = dataframe.loc[venue_no, 'name'] 
print('Latitude and longitude values of {} are {}, {}.'.format(venue_name, 
                                                               venue_latitude, 
                                                               venue_longitude))

Latitude and longitude values of Mexican Consulate are 43.64894155562399, -79.37867590887817.


In [26]:
latitude1=venue_latitude
longitude1=venue_longitude
radius=700
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=1BMK0S0BM3WY0AYZDHI3LGVTPHBALBFWQG0U14W20T1TQNIP&client_secret=UZDRGEAKSNAXCYN54101V3NQ3U0KYPZ34QDUN2XTRAV3GQFB&ll=43.64894155562399,-79.37867590887817&v=20200606&radius=700&limit=30'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ede50ab0cc1fd001b069f2f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-52138db911d22803b334c641-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '52138db911d22803b334c641',
       'location': {'address': 'Commerce Court East',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 87,
        'formattedAddress': ['Commerce Court East', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.64815919753501,
          'lng': -79.37874450467258}],


In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Mos Mos Coffee,Café,43.648159,-79.378745
1,Equinox Bay Street,Gym,43.648100,-79.379989
2,Brick Street Bakery,Bakery,43.648815,-79.380605
3,Pilot Coffee Roasters,Coffee Shop,43.648835,-79.380936
4,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323


In [30]:
print('In all there are {} venues near ''Quesada Mexican Grill'' within 700 meters'.format(nearby_venues.shape[0]))
print('In all there are {} categories of venues near ''Quesada Mexican Grill'' within 700 meters'.format(nearby_venues.groupby('categories').count().shape[0]))
nearby_venues.groupby('categories').count()['name']

In all there are 30 venues near Quesada Mexican Grill within 700 meters
In all there are 23 categories of venues near Quesada Mexican Grill within 700 meters


categories
American Restaurant              2
Bakery                           1
Bookstore                        1
Café                             5
Coffee Shop                      2
Creperie                         1
Deli / Bodega                    1
Food Truck                       1
Fountain                         1
Gastropub                        1
Gym                              2
Gym / Fitness Center             1
Hotel                            1
Ice Cream Shop                   1
Museum                           1
New American Restaurant          1
Park                             1
Pub                              1
Restaurant                       1
Seafood Restaurant               1
Tailor Shop                      1
Tea Room                         1
Vegetarian / Vegan Restaurant    1
Name: name, dtype: int64

## Find all the venues within 600 meters of each of the neighborhoods in Borough having word Toronto in their name.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
toronto_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [33]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [34]:
print(toronto_venues.shape)

(520, 7)


### check how many venues were returned for each neighborhood

In [35]:
toronto_venues.groupby(['Neighborhood']).count()['Venue']

Neighborhood
Berczy Park                                                                                                   30
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport    18
Central Bay Street                                                                                            30
Christie                                                                                                      17
Church and Wellesley                                                                                          30
Commerce Court, Victoria Hotel                                                                                30
First Canadian Place, Underground city                                                                        30
Garden District, Ryerson                                                                                      30
Harbourfront East, Union Station, Toronto Islands                                  

### Above table shows the number of venues available for each of the Neighborhoods in selected Borough (Names having word Toronto)

### Check out how many unique categories can be curated from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 149 uniques categories.


In [37]:
toronto_venues.groupby(['Venue Category']).count()['Venue']

Venue Category
Airport                           1
Airport Food Court                1
Airport Gate                      1
Airport Lounge                    2
Airport Service                   3
Airport Terminal                  2
American Restaurant               6
Aquarium                          1
Art Gallery                       7
Art Museum                        1
Arts & Crafts Store               3
Asian Restaurant                  2
BBQ Joint                         1
Baby Store                        1
Bakery                           12
Bank                              1
Bar                               5
Basketball Stadium                2
Beer Bar                          8
Belgian Restaurant                1
Bistro                            2
Boat or Ferry                     2
Bookstore                         5
Breakfast Spot                    4
Brewery                           1
Bubble Tea Shop                   4
Burger Joint                      5
Burrito Place

# Analyze Each Neighborhood

In [38]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [39]:
toronto_onehot.shape

(520, 149)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.0,0.000000,0.033333,0.033333,0.0,0.033333,0.000000,0.000000,0.033333,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.000000,0.0,0.033333,0.000000,0.0,0.000000,0.066667,0.066667,0.0,0.0,0.0,0.000000,0.033333,0.0,0.033333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.0,0.033333,0.000000,0.033333,0.033333,0.033333,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.033333,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.066667,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.055556,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.055556,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.055556,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000

In [41]:
toronto_grouped.shape

(19, 149)

### Print each neighborhood along with the top 10 most common venues

In [42]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2         Coffee Shop  0.07
3          Restaurant  0.03
4                Park  0.03
5                Café  0.03
6         Cheese Shop  0.03
7        Concert Hall  0.03
8            Creperie  0.03
9      Farmers Market  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.17
1          Coffee Shop  0.11
2        Boat or Ferry  0.11
3       Airport Lounge  0.11
4     Airport Terminal  0.11
5              Airport  0.06
6     Sculpture Garden  0.06
7                Plane  0.06
8  Rental Car Location  0.06
9      Harbor / Marina  0.06


----Central Bay Street----
                        venue  freq
0                 Coffee Shop  0.20
1                        Café  0.07
2                 Yoga Studio  0.03
3  Modern European Restaurant  0.03
4          Seafood Restaura

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### create the new dataframe and display the top 10 venues for each neighborhood.

In [44]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Bakery,Park,Concert Hall,Creperie,Café,Liquor Store,Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Coffee Shop,Boat or Ferry,Airport Lounge,Airport Terminal,Harbor / Marina,Rental Car Location,Sculpture Garden,Plane,Airport Food Court
2,Central Bay Street,Coffee Shop,Café,Yoga Studio,Poke Place,Bar,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Bubble Tea Shop,Comic Shop
3,Christie,Grocery Store,Café,Park,Playground,Coffee Shop,Nightclub,Candy Store,Italian Restaurant,Diner,Restaurant
4,Church and Wellesley,Burger Joint,Ramen Restaurant,Bubble Tea Shop,Restaurant,Juice Bar,Breakfast Spot,Salon / Barbershop,Bookstore,Pub,Dance Studio
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Gastropub,Deli / Bodega,Restaurant,American Restaurant,Steakhouse,Museum,Pub,Bookstore
6,"First Canadian Place, Underground city",Café,Coffee Shop,Tea Room,Restaurant,Seafood Restaurant,Sandwich Place,Asian Restaurant,Deli / Bodega,Pizza Place,Ice Cream Shop
7,"Garden District, Ryerson",Theater,Coffee Shop,Café,Modern European Restaurant,Electronics Store,Ramen Restaurant,Comic Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Hotel,Performing Arts Venue,Brewery,Bubble Tea Shop,Roof Deck,Salad Place,Japanese Restaurant,Lake
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Comfort Food Restaurant,Record Shop,Caribbean Restaurant,Cheese Shop,Cocktail Bar


# Cluster Neighborhoods

In [45]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 3, 1, 1, 1, 3, 1], dtype=int32)

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Playground,Trail,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Creperie
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,3,Park,Gastropub,Café,Italian Restaurant,Japanese Restaurant,Restaurant,Pet Store,Diner,Indian Restaurant,Jewelry Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Burger Joint,Ramen Restaurant,Bubble Tea Shop,Restaurant,Juice Bar,Breakfast Spot,Salon / Barbershop,Bookstore,Pub,Dance Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Bakery,Yoga Studio,Italian Restaurant,Performing Arts Venue,Chocolate Shop,Café,Pub
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Theater,Coffee Shop,Café,Modern European Restaurant,Electronics Store,Ramen Restaurant,Comic Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

### Cluster 1

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,0,Airport Service,Coffee Shop,Boat or Ferry,Airport Lounge,Airport Terminal,Harbor / Marina,Rental Car Location,Sculpture Garden,Plane,Airport Food Court


### Cluster 2

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,1,Coffee Shop,Park,Breakfast Spot,Bakery,Yoga Studio,Italian Restaurant,Performing Arts Venue,Chocolate Shop,Café,Pub
4,Downtown Toronto,1,Theater,Coffee Shop,Café,Modern European Restaurant,Electronics Store,Ramen Restaurant,Comic Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant
5,Downtown Toronto,1,Gastropub,Café,Restaurant,Coffee Shop,Italian Restaurant,Farmers Market,Diner,Creperie,Poke Place,Cosmetics Shop
6,Downtown Toronto,1,Seafood Restaurant,Coffee Shop,Cocktail Bar,Bakery,Park,Concert Hall,Creperie,Café,Liquor Store,Restaurant
7,Downtown Toronto,1,Coffee Shop,Café,Yoga Studio,Poke Place,Bar,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Bubble Tea Shop,Comic Shop
8,Downtown Toronto,1,Café,Coffee Shop,Asian Restaurant,Bakery,Seafood Restaurant,Restaurant,Plaza,Pizza Place,Opera House,Concert Hall
10,Downtown Toronto,1,Coffee Shop,Café,Deli / Bodega,Japanese Restaurant,Restaurant,Museum,Pizza Place,Sandwich Place,Beer Bar,Pub
11,Downtown Toronto,1,Café,Coffee Shop,Gastropub,Deli / Bodega,Restaurant,American Restaurant,Steakhouse,Museum,Pub,Bookstore
13,Downtown Toronto,1,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Comfort Food Restaurant,Record Shop,Caribbean Restaurant,Cheese Shop,Cocktail Bar
15,Downtown Toronto,1,Seafood Restaurant,Cocktail Bar,Café,Restaurant,Beer Bar,Farmers Market,Park,Museum,Comfort Food Restaurant,Concert Hall


### Cluster 3

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Playground,Coffee Shop,Nightclub,Candy Store,Italian Restaurant,Diner,Restaurant


### Cluster 4

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,3,Park,Gastropub,Café,Italian Restaurant,Japanese Restaurant,Restaurant,Pet Store,Diner,Indian Restaurant,Jewelry Store
2,Downtown Toronto,3,Burger Joint,Ramen Restaurant,Bubble Tea Shop,Restaurant,Juice Bar,Breakfast Spot,Salon / Barbershop,Bookstore,Pub,Dance Studio
9,Downtown Toronto,3,Park,Plaza,Hotel,Performing Arts Venue,Brewery,Bubble Tea Shop,Roof Deck,Salad Place,Japanese Restaurant,Lake
12,Downtown Toronto,3,Restaurant,Bookstore,Japanese Restaurant,Bakery,Yoga Studio,Music Venue,Sandwich Place,Beer Bar,Café,Park
18,Downtown Toronto,3,Coffee Shop,Park,Burger Joint,Sushi Restaurant,Ethiopian Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Martial Arts Dojo,Italian Restaurant,Hobby Shop


### Cluster 5

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Park,Playground,Trail,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Creperie
